# Debugging Numba problems



## Common problems

Numba is a compiler, if there's a problem, it could well be a "compilery" problem, the dynamic interpretation that comes with the Python interpreter is gone! As with any compiler toolchain there's a bit of a learning curve but once the basics are understood it becomes easy to write quite complex applications.

In [ ]:
from numba import njit
import numpy as np

### Type inference problems

A very large set of problems can be classed as type inference problems. These are problems which appear when Numba can't work out the types of all the variables in your code. Here's an example:

In [ ]:
@njit
def type_inference_problem():
    a = {}
    return a

type_inference_problem()

Things to note in the above, Numba has said that:
    
    1. It has encountered a typing error.
    2. It cannot infer (work out) the type of the variable named `a`.
    3. It has an imprecise type for `a` of `DictType[undefined, undefined]`.
    4. It's pointing to where the problem is in the source
    5. It's giving you things to look at for help
    
Numba's response is reasonable, how can it possibly compile a specialisation of an empty dictionary, it cannot work out what to use for a key or value type.

### Type unification problems

Another common issue is that of type unification, this is due to Numba needing the inferred variable types for the code it's compiling to be statically determined and type stable. What this usually means is something like the type of a variable is being changed in a loop or there's two (or more) possible return types. Example:

In [ ]:
@njit
def foo(x):
    if x > 10:
        return (1,)
    else:
        return 1

foo(1)

Things to note in the above, Numba has said that:
    
    1. It has encountered a typing error.
    2. It cannot unify the return types and then lists the offending types.
    3. It pointis to the locations in the source that are the cause of the problem.
    4. It's giving you things to look at for help.
    
Numba's response due to it not being possible to compile a function that returns a tuple or an integer? You couldn't do that in C/Fortran, same here!

### Unsupported features

Numba supports a subset of Python and NumPy, it's possible to run into something that hasn't been implemented. For example `str(int)` has not been written yet (this is a rather tricky thing to write :)). This is what it looks like:

In [ ]:
@njit
def foo():
    return str(10)

foo()

Things to note in the above, Numba has said that:
    
    1. It has encountered a typing error.
    2. It's an invalid use of a `Function` of type `(<class 'str'>)` with argument(s) of type(s): `(Literal[int](10))`
    3. It points to the location in the source that is the cause of the problem.
    4. It's giving you things to look at for help.


What's this bit about?
```
* parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
In definition 2:
    All templates rejected with literals.
In definition 3:
    All templates rejected without literals.
```
    
Internally Numba does something akin to "template matching" to try and find something to do the functionality requested with the types requested, it's looking through the definitions see if any match and reporting what they say (which in this case is "rejected").

Here's a different one, Numba's `np.mean` implementation doesn't support `axis`:

In [ ]:
@njit
def foo():
    x = np.arange(100).reshape((10, 10))
    return np.mean(x, axis=1)

foo()

Things to note in the above, Numba has said that:
    
    1. It has encountered a typing error.
    2. It's an invalid use of a `Function` "mean" with argument(s) of type(s): `(array(float64, 2d, C), axis=Literal[int](1))`
    3. It's reporting what the various template defintions are responding with: e.g. 
    "TypingError: numba doesn't support kwarg for mean", which is correct!
    4. It points to the location in the source that is the cause of the problem.
    5. It's giving you things to look at for help.
    
A common workaround for the above is to just unroll the loop over the axis, for example:

In [ ]:
@njit
def foo():
    x = np.arange(100).reshape((10, 10))
    lim, _ = x.shape
    buf = np.empty((lim,), x.dtype)
    for i in range(lim):
        buf[i] = np.mean(x[i])
    return buf
        
foo()

### Lowering errors

"Lowering" is the process of translating the Numba IR to LLVM IR to machine code. Numba tries really hard to prevent lowering errors, but sometimes you might see them, if you do please tell us:
    
https://github.com/numba/numba/issues/new

A lowering error means that there's a problem in Numba internals. The most common cause is that it worked out that it could compile a function as all the variable types were statically determined, but when it tried to find an implementation for some operation in the function to translate to machine code, it couldn't find one.

<h3><span style="color:blue"> Task 1: Debugging practice</span></h3>

The following code has a couple of issues, see if you can work them out and fix them.

In [ ]:
x = np.arange(20.).reshape((4, 5))

@njit
def problem_factory(x):
    nrm_x = np.linalg.norm(x, ord=2, axis=1) # axis not supported, manual unroll
    nrm_total = np.sum(nrm_x)
    ret = {} # dict type requires float->int cast, true branch is int and it sets the dict type
    if nrm_total > 87:
        ret[nrm_total] = 1
    else:
        ret[nrm_total] = nrm_total
        
    return ret


# This is a fixed version
@njit
def problem_factory_fixed(x):
    lim, _ = x.shape
    
    nrm_x = np.empty(lim, x.dtype)
    for i in range(lim):
        nrm_x[i] = np.linalg.norm(x[i])
    nrm_total = np.sum(nrm_x)

    ret = {}
    if nrm_total > 87:
        ret[nrm_total] = 1.0
    else:
        ret[nrm_total] = nrm_total
        
    return ret    

fixed = problem_factory_fixed(x)
expected = problem_factory.py_func(x)

# will pass if "fixed" correctly
for k, v in zip(fixed.items(), expected.items()):
    np.testing.assert_allclose(k[0], k[1])
    np.testing.assert_allclose(v[0], v[1])